자동차 정보 -> 가격예측

In [72]:
import pandas as pd

# 데이터 불러오기
train = pd.read_csv('C:/BigData/pythonstudy-main/pythonstudy-main/DA/train.csv')
test = pd.read_csv('C:/BigData/pythonstudy-main/pythonstudy-main/DA/test.csv')

# 데이터 확인
train.info()
print(train.describe())
train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7497 entries, 0 to 7496
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        7497 non-null   object 
 1   제조사       7497 non-null   object 
 2   모델        7497 non-null   object 
 3   차량상태      7497 non-null   object 
 4   배터리용량     4786 non-null   float64
 5   구동방식      7497 non-null   object 
 6   주행거리(km)  7497 non-null   int64  
 7   보증기간(년)   7497 non-null   int64  
 8   사고이력      7497 non-null   object 
 9   연식(년)     7497 non-null   int64  
 10  가격(백만원)   7497 non-null   float64
dtypes: float64(2), int64(3), object(6)
memory usage: 644.4+ KB
             배터리용량       주행거리(km)      보증기간(년)        연식(년)      가격(백만원)
count  4786.000000    7497.000000  7497.000000  7497.000000  7497.000000
mean     69.397187   44287.979458     4.960918     0.222489    62.331949
std      15.283635   55204.064386     3.155342     0.569232    36.646759
min      46.000000       3.000

,ID,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
0,TRAIN_0000,P사,TayGTS,Nearly New,86.077,AWD,13642,0,No,2,159.66
1,TRAIN_0001,K사,Niro,Nearly New,56.000,FWD,10199,6,No,0,28.01
2,TRAIN_0002,A사,eT,Brand New,91.200,AWD,2361,7,No,0,66.27
3,TRAIN_0003,A사,RSeTGT,Nearly New,NaN,AWD,21683,3,No,0,99.16
4,TRAIN_0004,B사,i5,Pre-Owned,61.018,AWD,178205,1,No,0,62.02


In [73]:
# 훈련 데이터셋 독립변수, 종속변수 분리
train_y = train['가격(백만원)']
train_x = train.drop(columns=['ID', '가격(백만원)'])

# 테스트 데이터셋 ID 제거 (keep -> 결과 csv 파일 첨부)
id_x = test['ID']
test_x = test.drop(columns='ID')

# train, test shape 열 개수 일치 확인
print(train_x.shape)
print(test_x.shape)

(7497, 9)
(846, 9)


In [74]:
# train 데이터셋 대상
# 칼럼 별 결측치 확인
columns = train_x.columns
for i in columns:
    print(i, train_x[i].isnull().sum())

# 배터리용량에 결측치 발견!

제조사 0
모델 0
차량상태 0
배터리용량 2711
구동방식 0
주행거리(km) 0
보증기간(년) 0
사고이력 0
연식(년) 0


In [75]:
# test 데이터셋 대상
# 칼럼 별 결측치 확인
columns = test_x.columns
for i in columns:
    print(i, test_x[i].isnull().sum())

# 배터리용량에 결측치 발견!

제조사 0
모델 0
차량상태 0
배터리용량 304
구동방식 0
주행거리(km) 0
보증기간(년) 0
사고이력 0
연식(년) 0


In [76]:
# 배터리용량은 수치형 데이터 -> 평균 대치 가능
print(train_x['배터리용량'].dtype)

# 결측치 대체 (평균 대치)
train_x['배터리용량'].fillna(train_x['배터리용량'].mean(), inplace=True)
test_x['배터리용량'].fillna(test_x['배터리용량'].mean(), inplace=True)

# 결측치 처리 확인 (전부 완료)
print(train_x['배터리용량'].isna().sum())
print(test_x['배터리용량'].isna().sum())

float64
0
0


In [77]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# 각 칼럼 당 unique 값의 개수 구하기
columns = train_x.columns
for i in columns:
    print({i}, len(train_x[i].unique()))

# 수치형과 명목형을 나눠서 라벨링이 아니라, unique값의 개수에 따라 나눠서 라벨링???
print(train_x.info())


# 질문!
# 수치형: 배터리용량, 주행거리, 보증기간, 연식
# unique값 다수보유: 배터리용량, 주행거리

# unique값이 작다면, 범주형으로 봐도 무관??

{'제조사'} 7
{'모델'} 21
{'차량상태'} 3
{'배터리용량'} 195
{'구동방식'} 3
{'주행거리(km)'} 6916
{'보증기간(년)'} 11
{'사고이력'} 2
{'연식(년)'} 3
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7497 entries, 0 to 7496
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   제조사       7497 non-null   object 
 1   모델        7497 non-null   object 
 2   차량상태      7497 non-null   object 
 3   배터리용량     7497 non-null   float64
 4   구동방식      7497 non-null   object 
 5   주행거리(km)  7497 non-null   int64  
 6   보증기간(년)   7497 non-null   int64  
 7   사고이력      7497 non-null   object 
 8   연식(년)     7497 non-null   int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 527.3+ KB
None


In [78]:
# 인코딩 / 정규화
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# 1. 라벨 인코딩 (명목형)
label_col = ['제조사', '모델', '차량상태', '구동방식', '보증기간(년)', '사고이력', '연식(년)']
encoder = LabelEncoder()
for c in label_col:
    train_x[c] = encoder.fit_transform(train_x[c])
    test_x[c] = encoder.fit_transform(test_x[c])

# 2. Min Max 정규화 (수치형)
scaler_col = ['배터리용량', '주행거리(km)']
scaler = MinMaxScaler()
# 따로 생성한 후, concat으로 붙이기
temp = train_x[scaler_col]
temp = pd.DataFrame(scaler.fit_transform(temp))
train_x = pd.concat([train_x, temp], axis=1).rename(columns= {0:'배터리용량_n',
                                                              1:'주행거리(km)_n'})
temp = test_x[scaler_col]
temp = pd.DataFrame(scaler.fit_transform(temp))
test_x = pd.concat([test_x, temp], axis=1).rename(columns={0:'배터리용량_n',
                                                           1: '주행거리(km)_n'})


In [79]:
# 정규화 이전 칼럼, 정규화 이후 칼럼 동시 존재
train_x.head()
test_x.head()

,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),배터리용량_n,주행거리(km)_n
0,4,15,1,76.093000,0,14057,2,0,0,0.559349,0.070294
1,1,20,0,90.000000,0,7547,8,0,0,0.817844,0.037735
2,1,19,0,69.712963,2,7197,7,1,0,0.440761,0.035985
3,2,2,1,68.479000,0,10357,7,0,1,0.417825,0.051789
4,3,0,0,69.712963,1,7597,10,0,0,0.440761,0.037985


In [80]:
# 정규화 이전 칼럼 삭제
train_x.drop(columns=scaler_col, inplace=True)
test_x.drop(columns=scaler_col, inplace=True)

# 정규화 최종 결과 확인
train_x.head()
test_x.head()

,제조사,모델,차량상태,구동방식,보증기간(년),사고이력,연식(년),배터리용량_n,주행거리(km)_n
0,4,15,1,0,2,0,0,0.559349,0.070294
1,1,20,0,0,8,0,0,0.817844,0.037735
2,1,19,0,2,7,1,0,0.440761,0.035985
3,2,2,1,0,7,0,1,0.417825,0.051789
4,3,0,0,1,10,0,0,0.440761,0.037985


In [85]:
# train 데이터셋 분할
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2, random_state=2025)

# 행 수 동일 확인 완료
print(X_train.shape)
print(y_train.shape)

(5997, 9)
(5997,)


In [108]:
import sklearn
print(dir(sklearn))

import sklearn.ensemble
print(dir(sklearn.ensemble))

['__SKLEARN_SETUP__', '__all__', '__builtins__', '__cached__', '__check_build', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', '_config', '_distributor_init', '_loss', 'base', 'clone', 'config_context', 'decomposition', 'dummy', 'ensemble', 'exceptions', 'get_config', 'linear_model', 'logger', 'logging', 'metrics', 'model_selection', 'neighbors', 'os', 'preprocessing', 'random', 'set_config', 'setup_module', 'show_versions', 'svm', 'sys', 'tree', 'utils']
['AdaBoostClassifier', 'AdaBoostRegressor', 'BaggingClassifier', 'BaggingRegressor', 'BaseEnsemble', 'ExtraTreesClassifier', 'ExtraTreesRegressor', 'GradientBoostingClassifier', 'GradientBoostingRegressor', 'IsolationForest', 'RandomForestClassifier', 'RandomForestRegressor', 'RandomTreesEmbedding', 'StackingClassifier', 'StackingRegressor', 'VotingClassifier', 'VotingRegressor', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package_

In [106]:
import sklearn.metrics
print(dir(sklearn.metrics))

['ConfusionMatrixDisplay', 'DetCurveDisplay', 'PrecisionRecallDisplay', 'RocCurveDisplay', 'SCORERS', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_base', '_classification', '_pairwise_fast', '_plot', '_ranking', '_regression', '_scorer', 'accuracy_score', 'adjusted_mutual_info_score', 'adjusted_rand_score', 'auc', 'average_precision_score', 'balanced_accuracy_score', 'brier_score_loss', 'calinski_harabasz_score', 'check_scoring', 'classification_report', 'cluster', 'cohen_kappa_score', 'completeness_score', 'confusion_matrix', 'consensus_score', 'coverage_error', 'davies_bouldin_score', 'dcg_score', 'det_curve', 'euclidean_distances', 'explained_variance_score', 'f1_score', 'fbeta_score', 'fowlkes_mallows_score', 'get_scorer', 'hamming_loss', 'hinge_loss', 'homogeneity_completeness_v_measure', 'homogeneity_score', 'jaccard_score', 'label_ranking_average_precision_score', 'label_ranking_loss', 'log_los

In [150]:
# 모델 생성 및 train 데이터셋으로 학습
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
model = RandomForestRegressor(n_estimators=1000, min_samples_leaf=1)
model.fit(X_train, y_train)
model2 = ExtraTreesRegressor()
model2.fit(X_train, y_train)

ExtraTreesRegressor()

In [151]:
from sklearn.metrics import mean_squared_error
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(mse)

y_pred2 = model2.predict(X_test)
mse2 = mean_squared_error(y_test, y_pred2)
print(mse2)

1.8689447647179989
2.2532238556153206


In [157]:
# 첫번째 모델 점수가 더 낮음 -> 오차율 적음 -> 첫번째 모델 선택
# 모델 이용한 test 데이터셋 예측
pred_y = pd.DataFrame(model.predict(test_x))

In [159]:
# 최종 DataFrame 생성 (id + predice data) 및 csv 파일 저장
result = pd.concat([id_x, pred_y], axis=1).rename(columns = {0:'predict_price'})
result.to_csv('C:/BigData/pythonstudy-main/pythonstudy-main/DA/result/carResult.csv')